In [25]:
import pandas as pd
import numpy as np
import json 
import copy
import csv
import os
import re
import nltk
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
from nltk.corpus import stopwords
import sklearn
from ngram import *


In [26]:
data = [] 
cf = open('bars_rev_tr_df.csv','r')

file = csv.DictReader(cf)
#print(file.fieldnames)

for x in file:
    line = {'business_id':x['business_id'],'date':x['date'],'stars':x['stars'],'text':x['text']}
    data.append(line)

cf.close()

In [27]:
data=data[0:100] 

In [28]:
reviewslist = copy.deepcopy(data)

In [53]:
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\$)|(\*)|(\%)|(\_)|(\=)|(\#)|(\&)|(\~)|(\@)")#[^\P{P}-]+
REPLACE_WITH_SPACE = re.compile("(\n)|(\-)|(\/)|(\d)")

def preprocess_reviews(reviews):
    reviews = REPLACE_NO_SPACE.sub("", reviews)
    reviews = REPLACE_WITH_SPACE.sub(" ", reviews)
    return reviews

reviewsTEXT_clean = copy.deepcopy(reviewslist)

for ind in range(len(reviewslist)):
    texts = ''
    texts = reviewslist[ind]['text']
    texts = texts.lower()
    
    texts = re.sub('n\'t',' not', texts)
    
    texts = re.sub('isnt','isn\'t', texts)
    texts = re.sub('wasnt','wasn\'t', texts)
    texts = re.sub('werent','weren\'t', texts)
    texts = re.sub('dont','don\'t', texts)
    texts = re.sub('doesnt','doesn\'t', texts)
    texts = re.sub('didnt','didn\'t', texts)
    texts = re.sub('hasnt','hasn\'t', texts)
    texts = re.sub('havent','haven\'t', texts)
    texts = re.sub('hadnt','hadn\'t', texts)
    texts = re.sub('mightnt','mightn\'t', texts)
    texts = re.sub('shouldnt','shouldn\'t', texts)
    texts = re.sub('isn','isn\'t', texts)
    texts = re.sub('wasn','wasn\'t', texts)
    texts = re.sub('weren','weren\'t', texts)
    texts = re.sub('don','don\'t', texts)
    texts = re.sub('doesn','doesn\'t', texts)
    texts = re.sub('didn','didn\'t', texts)
    texts = re.sub('hasn','hasn\'t', texts)
    texts = re.sub('haven','haven\'t', texts)
    texts = re.sub('hadn','hadn\'t', texts)
    texts = re.sub('mightn','mightn\'t', texts)
    texts = re.sub('shouldn','shouldn\'t', texts)
    
    texts = re.sub('n\'t',' not', texts)
    
    #add NOT_
    pattern = r'\.|\;|\!|\?|\,|\)|\(|\:|\'|\"|\%'
    list_text=re.split(pattern,texts)
    
    sent=''
    for i in range(len(list_text)):
        list_text[i] = re.sub('\+','', list_text[i])
        list_text[i] = re.sub('\*','', list_text[i])
        list_text[i] = re.sub('\$','', list_text[i])
        list_text[i] = re.sub('\[','', list_text[i])
        list_text[i] = re.sub('\]','', list_text[i])
        list_text[i] = re.sub('\%','', list_text[i])
        list_text[i] = re.sub('\\\\',' ', list_text[i])
        matchObj1 = re.search(r'(.*)not (.*)',list_text[i])
        matchObj2 = re.search(r'(.*) never (.*)',list_text[i])
        
        if matchObj1 != None :
            sub=re.sub(r' ', " notadd", ' '+matchObj1.group(2))
            list_text[i] = re.sub(matchObj1.group(2) ,sub, list_text[i])
            list_text[i] = re.sub(r'not ','', list_text[i],1)
            #list_text[i] = re.sub(',',' ', matchObj1.group(3))
            sent = sent + list_text[i] + ' '
        
        elif matchObj2 != None :
            sub=re.sub(r' ', " notadd", ' '+matchObj2.group(2))
            list_text[i] = re.sub(matchObj2.group(2) ,sub, list_text[i])
            list_text[i] = re.sub(r'never ','', list_text[i],1)
            #list_text[i] = re.sub(',',' ', matchObj1.group(3))
            sent = sent + list_text[i] + ' '
        
        else:
            sent = sent + list_text[i] + ' '
   
    #print(sent)
    #print(texts)
    #print(texts)
    text_clean=preprocess_reviews(sent)
    #print(text_clean)
    reviewsTEXT_clean[ind]['text']=text_clean
    reviewsTEXT_clean[ind]['text_split']=text_clean.split()
    reviewsTEXT_clean[ind]['bigrams']=generate_ngrams(text_clean, 3)
    reviewsTEXT_clean[ind]['freq']=nltk.FreqDist(reviewsTEXT_clean[ind]['text_split'])
    reviewsTEXT_clean[ind]['bigrams_freq']=nltk.FreqDist(reviewsTEXT_clean[ind]['bigrams'])
    
#reviewsTEXT_clean[0]

In [55]:
stop_words = set(stopwords.words('english')) 
negWords=set(["wouldn't",'isn','wasn',"weren't", "haven't", "hasn't", "couldn't", "isn't", 'doesn','hasn',"mustn't", 'mightn', 'shan', 'no', "wasn't",'aren', "didn't", "hadn't","don't",'nor',"won't",'weren',"doesn't","needn't", 'shouldn',"mightn't","shan't", 'wouldn',"shouldn't",'hadn'])
d=set(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','à','{sigh}'])
stopWords=stop_words-negWords
stopWords.update(d)


In [56]:
#select bigrams due to frequency order
AllBiFreq = reviewsTEXT_clean[0]['bigrams_freq']
for ind in range(len(reviewsTEXT_clean)-1):
    Bifreqs = reviewsTEXT_clean[ind+1]['bigrams_freq']
    AllBiFreq.update(Bifreqs)
    
BiwordList = sorted(AllBiFreq.items(),key=lambda item:item[1],reverse=True)

#choose the important ones--No.1-100
BiWORDS=[x[0] for x in BiwordList]

In [57]:
Biwordfreq = {}
for i in range(len(BiwordList)):
    if len(list(set(BiwordList[i][0].split()).difference(stopWords))) >= 2:
        Biwordfreq[BiwordList[i][0]]=BiwordList[i][1]

In [59]:
#create list of texts
texts= [x['text'] for x in reviewsTEXT_clean]

In [61]:
#vectorizer = CountVectorizer(token_pattern=r'\b[^\d\W]+\b')
vectorizer = CountVectorizer(ngram_range=(2, 3))
#X = vectorizer.fit_transform(texts)
transformer = TfidfTransformer()

tfidf = transformer.fit_transform(vectorizer.fit_transform(texts))

X=tfidf

In [78]:
X_df=pd.DataFrame(X.A,columns = vectorizer.get_feature_names())


In [80]:
Reviews_df=X_df[list(set(Biwordfreq.keys()).intersection(set(vectorizer.get_feature_names())))]


In [81]:
Reviews_df

,could have tiramisu,time are usually,great for families,two weeks,the tacos either,food is notaddshow,notaddjeans notaddand notadda,wait for preparation,table although waiting,notaddbekke notaddis but,...,really pissed off,style pintos,tried other buffalo,had chocolate drizzled,ordered chicken tacos,first we changed,is pretty good,piping hot,cue sandwich,balanced sauce
0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
6,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
8,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
9,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
